In [1]:
import re
import glob
import xlwings as xw
import math
from pathlib import Path
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as mclr
import plotly.express as px
import statsmodels.api as sm
import pylab as py
import os, cv2, glob, tempfile
import joblib

from scipy import stats
from scipy.stats import pearsonr

import sklearn
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.datasets import make_blobs

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import metrics
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
make_scorer = sklearn.metrics.make_scorer
f1 = make_scorer(f1_score, pos_label=1, average="binary")
from sklearn.metrics import classification_report
from sklearn.inspection import permutation_importance
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import RepeatedStratifiedKFold

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras import utils as np_utils
from keras import models
import keras_tuner as kt
from scipy import signal

import config as config
print(config.__path__)
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras import backend as K

import argparse
# from sklearn.utils import class_weight
print(tf.__version__)
print(keras.__version__)

import shutil
import xlwings as xw

%matplotlib inline
mpl.use("Agg")

['C:\\Users\\user\\anaconda3\\lib\\site-packages\\config']
2.12.0-dev20221107
2.12.0


In [2]:
import warnings
warnings.filterwarnings("ignore")

# 1. Data Preparation

In [3]:
## input training set ## 90960 x 16 df
df_ROI_train = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\afterModelSelection\\cbMSIn_0d01\\df_train_cbMSIn0d01_norm_0n1_postPredict.csv").drop(columns=["Column1"]).reset_index().drop(columns=["index"])

In [4]:
## input ext val set ## 6075 x 16 df
df_ROI_ext = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\afterModelSelection\\cbMSIn_0d01\\df_ext_cbMSIn0d01_norm_0n1_postPredict.csv").drop(columns=["Column1"]).reset_index().drop(columns=["index"])

In [5]:
## input FNA set ## 88701 x 16 df
df_ROI_FNA = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\afterModelSelection\\cbMSIn_0d01\\df_FNA_cbMSIn0d01_norm_0n1_postPredict.csv").drop(columns=["Column1"]).reset_index().drop(columns=["index"])

In [6]:
## input DirectInfusion set ## 6057 x 16 df
df_ROI_Di = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\afterModelSelection\\cbMSIn_0d01\\df_Di_cbMSIn0d01_norm_0n1_postPredict.csv").drop(columns=["Column1"]).reset_index().drop(columns=["index"])

In [7]:
df_ROI_Di

,pixel_id,311.165,269.245,215.03,295.225,883.53,309.165,738.505,435.295,250.145,514.285,type,predicted0n1,T,F,CM
0,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.005209,0.142200,0.055132,0.003688,0.077996,0.211143,0.061852,0.125846,0.465556,0.036498,0,0,0,1,TN
1,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.089567,0.156323,0.173760,0.001074,0.167022,0.317430,0.127993,0.145506,0.581512,0.099618,0,1,1,0,FP
2,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.032961,0.134117,0.033342,-0.069846,0.121979,0.212942,0.028980,0.076804,0.463399,0.056825,0,0,0,1,TN
3,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.080976,0.235063,0.080277,0.057323,0.129092,0.303446,0.108110,0.192986,0.548904,0.090339,0,0,0,1,TN
4,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.132523,0.236850,0.151528,0.080082,0.239269,0.394202,0.236241,0.204536,0.643706,0.147333,0,0,0,1,TN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6052,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.002071,-0.209544,0.151144,-0.039347,-0.042523,-0.056643,0.023261,0.147144,0.000700,0.163267,1,1,1,0,TP
6053,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.311749,-0.218071,-0.309877,-0.299480,-0.273013,-0.303035,-0.280535,-0.277867,-0.027025,-0.251944,1,0,0,1,FN
6054,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.155841,-0.037302,-0.001820,-0.048717,-0.045113,-0.153138,-0.084896,-0.039435,-0.091159,0.091528,1,1,1,0,TP
6055,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.130135,-0.100402,0.064682,-0.108138,-0.006814,-0.127244,-0.068984,-0.070669,-0.003682,-0.018040,1,1,1,0,TP


In [8]:
df_ROI_train["Sample"] = 0
df_ROI_train["Row"] = 0
df_ROI_train["Scan"] = 0
df_ROI_train_Sample = []
df_ROI_train_Row = []
df_ROI_train_Scan = []
for i in range(len(df_ROI_train["pixel_id"])):
    df_ROI_train_Sample.append("_".join(str(df_ROI_train["pixel_id"][i]).split("_")[:-2]))
    df_ROI_train_Row.append(int(str(df_ROI_train["pixel_id"][i]).split("_")[-1]))
    df_ROI_train_Scan.append(int(str(df_ROI_train["pixel_id"][i]).split("_")[-2]))
df_ROI_train["Sample"] = df_ROI_train_Sample
df_ROI_train["Row"] = df_ROI_train_Row
df_ROI_train["Scan"] = df_ROI_train_Scan
pop_column = df_ROI_train.pop('Scan')
df_ROI_train.insert(1, 'Scan', pop_column)
pop_column = df_ROI_train.pop('Row')
df_ROI_train.insert(1, 'Row', pop_column)
pop_column = df_ROI_train.pop('Sample')
df_ROI_train.insert(1, 'Sample', pop_column)

df_ROI_ext["Sample"] = 0
df_ROI_ext["Row"] = 0
df_ROI_ext["Scan"] = 0
df_ROI_ext_Sample = []
df_ROI_ext_Row = []
df_ROI_ext_Scan = []
for i in range(len(df_ROI_ext["pixel_id"])):
    df_ROI_ext_Sample.append("_".join(str(df_ROI_ext["pixel_id"][i]).split("_")[:-2]))
    df_ROI_ext_Row.append(int(str(df_ROI_ext["pixel_id"][i]).split("_")[-1]))
    df_ROI_ext_Scan.append(int(str(df_ROI_ext["pixel_id"][i]).split("_")[-2]))
df_ROI_ext["Sample"] = df_ROI_ext_Sample
df_ROI_ext["Row"] = df_ROI_ext_Row
df_ROI_ext["Scan"] = df_ROI_ext_Scan
pop_column = df_ROI_ext.pop('Scan')
df_ROI_ext.insert(1, 'Scan', pop_column)
pop_column = df_ROI_ext.pop('Row')
df_ROI_ext.insert(1, 'Row', pop_column)
pop_column = df_ROI_ext.pop('Sample')
df_ROI_ext.insert(1, 'Sample', pop_column)

df_ROI_FNA["Sample"] = 0
df_ROI_FNA["Row"] = 0
df_ROI_FNA["Scan"] = 0
df_ROI_FNA_Sample = []
df_ROI_FNA_Row = []
df_ROI_FNA_Scan = []
for i in range(len(df_ROI_FNA["pixel_id"])):
    df_ROI_FNA_Sample.append("_".join(str(df_ROI_FNA["pixel_id"][i]).split("_")[:-2]))
    df_ROI_FNA_Row.append(int(str(df_ROI_FNA["pixel_id"][i]).split("_")[-1]))
    df_ROI_FNA_Scan.append(int(str(df_ROI_FNA["pixel_id"][i]).split("_")[-2]))
df_ROI_FNA["Sample"] = df_ROI_FNA_Sample
df_ROI_FNA["Row"] = df_ROI_FNA_Row
df_ROI_FNA["Scan"] = df_ROI_FNA_Scan
pop_column = df_ROI_FNA.pop('Scan')
df_ROI_FNA.insert(1, 'Scan', pop_column)
pop_column = df_ROI_FNA.pop('Row')
df_ROI_FNA.insert(1, 'Row', pop_column)
pop_column = df_ROI_FNA.pop('Sample')
df_ROI_FNA.insert(1, 'Sample', pop_column)

df_ROI_Di["Sample"] = 0
df_ROI_Di["Row"] = 0
df_ROI_Di["Scan"] = 0
df_ROI_Di_Sample = []
df_ROI_Di_Row = []
df_ROI_Di_Scan = []
for i in range(len(df_ROI_Di["pixel_id"])):
    df_ROI_Di_Sample.append("_".join(str(df_ROI_Di["pixel_id"][i]).split("_")[:-2]))
    df_ROI_Di_Row.append(int(str(df_ROI_Di["pixel_id"][i]).split("_")[-1]))
    df_ROI_Di_Scan.append(int(str(df_ROI_Di["pixel_id"][i]).split("_")[-2]))
df_ROI_Di["Sample"] = df_ROI_Di_Sample
df_ROI_Di["Row"] = df_ROI_Di_Row
df_ROI_Di["Scan"] = df_ROI_Di_Scan
pop_column = df_ROI_Di.pop('Scan')
df_ROI_Di.insert(1, 'Scan', pop_column)
pop_column = df_ROI_Di.pop('Row')
df_ROI_Di.insert(1, 'Row', pop_column)
pop_column = df_ROI_Di.pop('Sample')
df_ROI_Di.insert(1, 'Sample', pop_column)

In [9]:
df_ROI_train = df_ROI_train.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_ROI_ext = df_ROI_ext.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_ROI_FNA = df_ROI_FNA.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])
df_ROI_Di = df_ROI_Di.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])

In [10]:
df_ROI_Di

,pixel_id,Sample,Row,Scan,311.165,269.245,215.03,295.225,883.53,309.165,738.505,435.295,250.145,514.285,type,predicted0n1,T,F,CM
0,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,1,0.005209,0.142200,0.055132,0.003688,0.077996,0.211143,0.061852,0.125846,0.465556,0.036498,0,0,0,1,TN
1,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,400,0.055137,0.307273,0.127241,0.049571,0.115976,0.285165,0.139311,0.244449,0.547018,0.067742,0,0,0,1,TN
2,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,401,0.082894,0.122738,0.153728,-0.003172,0.155022,0.322064,0.093482,0.161298,0.562523,0.100333,0,1,1,0,FP
3,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,402,0.029711,0.280842,0.081626,0.045472,0.159267,0.206064,0.159571,0.261471,0.490588,0.043914,0,0,0,1,TN
4,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01,1,403,0.107092,0.286038,0.042775,0.063968,0.130098,0.344899,0.117790,0.177346,0.566772,0.094227,0,0,0,1,TN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6052,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12,1,203,-0.329523,-0.315993,-0.309877,-0.344407,-0.273013,-0.362302,-0.280535,-0.277867,-0.091159,-0.271914,1,1,1,0,TP
6053,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12,1,204,-0.205763,-0.111500,-0.175082,-0.189053,-0.273013,-0.181593,-0.199019,0.053256,-0.034563,-0.119012,1,0,0,1,FN
6054,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12,1,205,-0.074780,-0.036130,0.172757,-0.020823,-0.007824,-0.095070,-0.080780,0.036527,0.002191,-0.002107,1,1,1,0,TP
6055,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12,1,207,-0.326406,-0.259317,-0.309877,-0.313903,-0.273013,-0.294668,-0.280535,-0.277867,0.000815,-0.265437,1,0,0,1,FN


In [11]:
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI = df_ROI_train.loc[df_ROI_train["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI = df_ROI_ext.loc[df_ROI_ext["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI = df_ROI_Di.loc[df_ROI_Di["Sample"] == "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])


In [12]:
sample_ROI_list = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI, 
                   HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI, 
                   HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI, 
                   HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI, 
                   HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI, 
                   HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI]

In [13]:
name_list = ["HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI", 
             "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI", 
             "HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI"]

In [14]:
sample_ROI_list_train = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI, 
                         HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI, 
                         HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI]

sample_ROI_list_ext = [HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI, 
                       HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI, 
                       HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI, 
                       HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI, 
                       HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI]

sample_ROI_list_ingested = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3_ROI, 
                            HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4_ROI]

sample_ROI_list_FNA = [HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI, 
                       HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI, 
                       HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI, 
                       HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI]

sample_ROI_list_DirectIn = [HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI, 
                            HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI, 
                            HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI]

In [15]:
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI

,pixel_id,Sample,Row,Scan,311.165,269.245,215.03,295.225,883.53,309.165,738.505,435.295,250.145,514.285,type,predicted0n1,T,F,CM
0,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,4,0.160604,-0.163045,0.248452,0.023961,-0.112722,0.202780,-0.400633,-0.214542,0.051972,0.156889,0,1,1,0,FP
1,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,5,-0.006580,0.069130,0.240045,0.088069,-0.128179,0.189331,0.006154,0.035691,0.142108,0.116393,0,0,0,1,TN
2,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,6,-0.006580,0.101559,0.245815,0.045857,-0.153132,0.176822,-0.400633,0.070012,0.172701,0.157971,0,0,0,1,TN
3,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,194,0.232619,0.663471,0.444382,0.574303,0.307173,0.622925,0.197538,0.452585,0.621820,0.391120,0,0,0,1,TN
4,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,195,0.280382,0.603420,0.521236,0.639374,0.069512,0.633094,0.248378,0.354982,0.653565,0.495576,0,0,0,1,TN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14820,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,117,144,-0.006580,0.114175,0.120504,0.073202,0.089360,0.084981,-0.017698,0.253488,-0.058141,0.056678,0,0,0,1,TN
14821,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,117,145,-0.006580,0.110864,0.149278,0.138092,0.009676,-0.033384,-0.074096,0.195062,0.005663,0.104883,0,0,0,1,TN
14822,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,117,146,-0.006580,0.097747,0.174242,0.149853,-0.037486,-0.033384,0.048093,0.201845,-0.024107,0.059666,0,0,0,1,TN
14823,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,118,177,-0.006580,0.482791,-0.227219,0.283037,-0.041197,0.219788,-0.124367,0.076925,0.403174,-0.036019,0,0,0,1,TN


In [16]:
HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI

,pixel_id,Sample,Row,Scan,311.165,269.245,215.03,295.225,883.53,309.165,738.505,435.295,250.145,514.285,type,predicted0n1,T,F,CM
0,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_5_1,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,1,5,0.766507,0.209647,-0.227219,0.051280,0.104003,0.841147,-0.097980,-0.214542,0.359056,-0.036019,1,0,0,1,FN
1,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_6_1,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,1,6,0.614838,0.278745,-0.227219,-0.152279,0.101500,0.676736,-0.022809,-0.214542,0.418288,-0.036019,1,0,0,1,FN
2,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_7_1,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,1,7,0.572119,0.293238,-0.227219,0.048475,0.076880,0.584615,0.077995,-0.214542,0.475203,-0.036019,1,0,0,1,FN
3,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_8_1,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,1,8,0.596465,0.364807,-0.227219,0.053760,-0.102235,0.607139,0.040858,-0.214542,0.462246,-0.036019,1,0,0,1,FN
4,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_9_1,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,1,9,0.648145,0.313519,-0.227219,0.097808,-0.110454,0.685772,-0.004592,-0.214542,0.502965,-0.036019,1,0,0,1,FN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14249,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,125,189,0.200390,0.374286,-0.227219,0.214484,-0.019561,-0.033384,-0.400633,-0.214542,0.509440,-0.036019,1,0,0,1,FN
14250,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,125,190,0.205362,0.344505,-0.227219,0.175964,-0.048004,-0.033384,-0.400633,-0.214542,0.530814,-0.036019,1,0,0,1,FN
14251,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,125,191,0.201067,0.359337,-0.227219,0.194580,-0.045142,-0.033384,-0.400633,-0.214542,0.583072,-0.036019,1,0,0,1,FN
14252,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3,126,185,0.199028,0.388705,-0.227219,0.199541,0.048273,-0.033384,-0.400633,-0.214542,0.576478,-0.036019,1,0,0,1,FN


# 2. Prediction (made)

## 2.1. FNA

In [17]:
( list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI["predicted0n1"]).count(1)) ) *100, "%"

(75.13659359190557, '%')

In [18]:
( list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI["predicted0n1"]).count(1)) ) *100, "%"

(47.555832123349234, '%')

In [19]:
( list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI["predicted0n1"]).count(1)) ) *100, "%"

(19.971151324416468, '%')

In [20]:
( list(HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3_ROI["predicted0n1"]).count(1)) ) *100, "%"

(82.65749964922126, '%')

In [21]:
( list(HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5_ROI["predicted0n1"]).count(1)) ) *100, "%"

(67.0785035052405, '%')

In [22]:
( list(HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2_ROI["predicted0n1"]).count(1)) ) *100, "%"

(67.9483870967742, '%')

## 2.2. Direct Infusion

In [23]:
( list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S01_ROI["predicted0n1"]).count(1)) ) *100, "%"

(5.306799336650083, '%')

In [24]:
( list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S04_ROI["predicted0n1"]).count(1)) ) *100, "%"

(56.27062706270627, '%')

In [25]:
( list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S05_ROI["predicted0n1"]).count(1)) ) *100, "%"

(17.986798679867988, '%')

In [26]:
( list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S10_ROI["predicted0n1"]).count(1)) ) *100, "%"

(19.141914191419144, '%')

In [27]:
( list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_S12_ROI["predicted0n1"]).count(1)) ) *100, "%"

(69.14191419141915, '%')

In [28]:
( list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S01_ROI["predicted0n1"]).count(1)) ) *100, "%"

(41.914191419141915, '%')

In [29]:
( list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S04_ROI["predicted0n1"]).count(1)) ) *100, "%"

(60.726072607260726, '%')

In [30]:
( list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S10_ROI["predicted0n1"]).count(1)) ) *100, "%"

(54.290429042904286, '%')

In [31]:
( list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S11_ROI["predicted0n1"]).count(1)) ) *100, "%"

(63.20132013201321, '%')

In [32]:
( list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI["predicted0n1"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI["predicted0n1"]).count(0) + list(HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S12_ROI["predicted0n1"]).count(1)) ) *100, "%"

(84.65346534653465, '%')

# 3. EDA

In [33]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

In [34]:
import matplotlib.pyplot as plt
import matplotlib.colors
import matplotlib.cm
import numpy as np
from matplotlib.colors import ListedColormap
cmap = cm.binary

def rgb_white2alpha(rgb, ensure_increasing=False):

    # The most transparent alpha we can use is given by the min of RGB
    # Convert it from saturation to opacity
    alpha = 1. - np.min(rgb, axis=1)
    if ensure_increasing:
        # Let's also ensure the alpha value is monotonically increasing
        a_max = alpha[0]
        for i, a in enumerate(alpha):
            alpha[i] = a_max = np.maximum(a, a_max)
    alpha = np.expand_dims(alpha, -1)
    # Rescale colors to discount the white that will show through from transparency
    rgb = (rgb + alpha - 1) / alpha
    # Concatenate our alpha channel
    return np.concatenate((rgb, alpha), axis=1)
    

def cmap_white2alpha(name, ensure_increasing=False, register=True):

    # Fetch the cmap callable
    cmap = plt.get_cmap(name)
    # Get the colors out from the colormap LUT
    rgb = cmap(np.arange(cmap.N))[:, :3]  # N-by-3
    # Convert white to alpha
    rgba = rgb_white2alpha(rgb, ensure_increasing=ensure_increasing)
    # Create a new Colormap object
    cmap_alpha = matplotlib.colors.ListedColormap(rgba, name=name + "_alpha")
    if register:
        matplotlib.cm.register_cmap(name=name + "_alpha", cmap=cmap_alpha)
    return cmap_alpha

In [47]:
# Get our new version of the colormap with white2alpha
cmap_w2a = cmap_white2alpha("Reds")

In [48]:
def genAROItif(sample_ROI, sample_, name, mz, vmax_=1):

    df_heatmap = sample_ROI[["Row", "Scan", mz]]
    df_heatmap2 = pd.DataFrame(np.array([[0, 0, 0]]), columns=['Row', 'Scan', mz]).reset_index().drop(columns = ["index"])
    df_HM = df_heatmap2.loc[df_heatmap["Row"] == 0].loc[df_heatmap2["Scan"] == 0]
    Heat_Row = sample_["Row"]
    Heat_Scan = sample_["Scan"]


    #hi = 0
    for y in range(len(set(Heat_Row))):
        for x in range(1, len(set(Heat_Scan))+1):

            df_in = df_heatmap.loc[df_heatmap["Row"] == (y+1)].loc[df_heatmap["Scan"] == x]
            if len(df_in) == 1:
                df_HM = pd.concat([df_HM, df_in], join="inner", ignore_index=True)
                df_HM = df_HM.sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns = ["index"])
                #hi += 1


            elif len(df_in) == 0:
                df_heatmap2["Row"] = y+1
                df_heatmap2["Scan"] = x
                df_HM = pd.concat([df_HM, df_heatmap2], join="inner", ignore_index=True)
                df_HM = df_HM.sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns = ["index"])
                #hi += 1


    #print(name, mz, hi, len(df_HM))
    if "index" in df_HM.columns:
        df_HM = df_HM.drop(columns = ["index", "Row", "Scan"])
    else:
        df_HM = df_HM.drop(columns = ["Row", "Scan"])

    df_HM = df_HM.T
    New_y = []
    for y in range(1, len(set(Heat_Row))+1):
        New_y.append(str(y))
    New_x = []
    for x in range(1, len(set(Heat_Scan))+1):
        New_x.append(str(x))

    arr = df_HM.values.copy()
    arr.resize(len(set(Heat_Row)), len(set(Heat_Scan)))

    df_HM = pd.DataFrame(arr)
    df_HM.index = New_y
    df_HM.columns = New_x

    for i in list(df_HM.columns):
        df_HM[i] = df_HM[i].astype(float)

    plt.subplots(dpi = 300)
    plt.imshow(df_HM, cmap=cmap_w2a, vmin=0, vmax=1, zorder=1) #cmap_w2a
    plt.xticks([])
    plt.yticks([])
    plt.box(False)
    plt.autoscale(enable=True, tight=True)

    plt.rcParams['figure.figsize'] = [2, 2]

    plt.savefig(f"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/cbMSIn_0d01/{name}_{mz}_1.tif", bbox_inches = 'tight', pad_inches = 0)

In [37]:
df_raMSI_ML = pd.read_csv(r'H:/3_output_raMSIn/3_3_Output_raMSI_HKU_Ingested4FNA/0_cbMSI_0d01/cbMSI0d01_noLT.csv').set_index('pixel_id')
df_raMSI_ML.shape

(371480, 2390)

In [38]:
raMSI_ML = df_raMSI_ML.reset_index()

In [39]:
raMSI_ML["Sample"] = 0
raMSI_ML["Row"] = 0
raMSI_ML["Scan"] = 0
Sample = []
Row = []
Scan = []
for i in range(len(raMSI_ML["pixel_id"])):
    Sample.append("_".join(str(raMSI_ML["pixel_id"][i]).split("_")[:-2]))
    Row.append(int(str(raMSI_ML["pixel_id"][i]).split("_")[-1]))
    Scan.append(int(str(raMSI_ML["pixel_id"][i]).split("_")[-2]))
raMSI_ML["Sample"] = Sample
raMSI_ML["Row"] = Row
raMSI_ML["Scan"] = Scan

In [40]:
pop_column = raMSI_ML.pop('Scan')
raMSI_ML.insert(1, 'Scan', pop_column)
pop_column = raMSI_ML.pop('Row')
raMSI_ML.insert(1, 'Row', pop_column)
pop_column = raMSI_ML.pop('Sample')
raMSI_ML.insert(1, 'Sample', pop_column)

In [41]:
raMSI_ML = raMSI_ML.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])

In [42]:
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

In [43]:
sample_list = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4, 
               HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3, 
               HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6, 
               HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2]

In [49]:
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime

TOP ="T"
    

counter = 0

for im in range(len(sample_ROI_list[:-10])):
    start = datetime.now()

    name = name_list[im]
    sample_ = sample_list[im]
    preview = sample_ROI_list[im][[TOP]]
    for i in preview.columns:
        preview[i] = preview[i].astype(float)

    vmax_list = list(preview.describe().iloc[-1])
    vmax_ = 1
    #print(TOP, name, vmax_)

    genAROItif(sample_ROI=sample_ROI_list[im], sample_=sample_, name=name, mz=TOP, vmax_=vmax_)
    #print(df.shape)
    counter += 1
    end = datetime.now()
    Duration = end - start
    print(Duration)


0:00:47.514731
0:00:52.804857
0:01:01.046006
0:00:33.535319
0:00:39.593979
0:00:53.452031
0:00:03.470356
0:00:02.437702
0:00:03.819817
0:00:03.663427
0:01:54.019517
0:01:49.536280
0:01:57.943159
0:00:54.608765
0:00:46.022171
0:00:38.981810
0:00:38.607212
0:00:37.617755
0:00:38.449023
0:00:02.390263
0:00:05.820752
0:00:03.231704
0:00:02.777011
0:01:46.675798
0:01:48.361904
0:02:01.105550
